In [1]:
!nvidia-smi

Tue May 24 21:25:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
!pip install git+https://github.com/cene555/Imagen-pytorch-accelerate.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cene555/Imagen-pytorch-accelerate.git to /tmp/pip-req-build-qprjjzls
  Running command git clone -q https://github.com/cene555/Imagen-pytorch-accelerate.git /tmp/pip-req-build-qprjjzls
  Created wheel for imagen-pytorch: filename=imagen_pytorch-0.0.0-py3-none-any.whl size=1967219 sha256=0ff62a931705ac4a7652efc421d91e6c3dd0f00e7b084423adc3f41586cc89d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-hevyenz1/wheels/28/7b/f9/33e9437c65146ad77a7c1319a206e587ac3e4bc243a2ff7f5f
Successfully built imagen-pytorch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%capture
!git lfs install
!git clone https://huggingface.co/Cene655/Imagen-pytorch

In [5]:
%%capture
!unzip /content/drive/MyDrive/train2014.zip
!unzip /content/drive/MyDrive/annotations_trainval2014.zip

In [13]:
!pip uninstall imagen_pytorch --y

Found existing installation: imagen-pytorch 0.0.0
Uninstalling imagen-pytorch-0.0.0:
  Successfully uninstalled imagen-pytorch-0.0.0


In [1]:
from PIL import Image
from IPython.display import display
import torch as th
import pandas as pd
from imagen_pytorch.download import load_checkpoint
from imagen_pytorch.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)
from imagen_pytorch.resample import create_named_schedule_sampler

from imagen_pytorch import logger
from imagen_pytorch.dataset import get_loader
from imagen_pytorch.train_utils import TrainLoop

has_cuda = th.cuda.is_available()
device = th.device('cpu' if not has_cuda else 'cuda')

In [ ]:
options = model_and_diffusion_defaults()
options['use_fp16'] = False
options['t5_name'] = 't5-large'
model, diffusion = create_model_and_diffusion(**options)
model.eval()
#if has_cuda:
#    model.convert_to_fp16()
model.to(device)
#model.load_state_dict(load_checkpoint('base', device), strict=False)
model.load_state_dict(th.load('/content/Imagen-pytorch/imagen-pytorch.pt'))

print('total base parameters', sum(x.numel() for x in model.parameters()))

In [3]:
def get_images_id(images_list):
    images_dict = {}
    for i in images_list:
        images_dict[i['id']] = i['file_name']
    return images_dict

In [4]:
import json

with open('/content/annotations/captions_train2014.json') as json_file:
    data = json.load(json_file)

In [5]:
images_dict = get_images_id(data['images'])

In [6]:
df = []
for annotation in data['annotations']:
    df.append([images_dict[annotation['image_id']], annotation['caption']])

In [7]:
df = pd.DataFrame(df)
df.columns = ['path', 'text']

In [ ]:
data = get_loader(batch_size=4,
                  resolution=64,
                   image_dir='/content/train2014',
                   df=df,
                   tokenizer_name='t5-large', 
                   max_len=128,
                   zero_text_prob=0.1,
                   shuffle=True)

In [9]:
#a, b = next(data)

In [10]:
logger.configure()

logger.log("creating model and diffusion...")

Logging to /tmp/openai-2022-05-24-21-41-28-264075
creating model and diffusion...


In [11]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

In [ ]:
TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=4,
        microbatch=-1,
        lr=1e-4,
        ema_rate="0.9999",
        log_interval=100,
        save_interval=10000,
        resume_checkpoint=False,
        use_fp16=False,
        fp16_scale_growth=1e-3,
        schedule_sampler=schedule_sampler,
        weight_decay=0.01,
        lr_anneal_steps=0,
        save_dir='/content/',
).run_loop()

------------------------
| grad_norm | 0.0217   |
| loss      | 0.0207   |
| loss_q0   | 0.0646   |
| loss_q1   | 0.0136   |
| loss_q2   | 0.00235  |
| mse       | 0.0191   |
| mse_q0    | 0.0625   |
| mse_q1    | 0.0124   |
| mse_q2    | 0.000708 |
| samples   | 4        |
| step      | 0        |
| vb        | 0.00162  |
| vb_q0     | 0.00203  |
| vb_q1     | 0.00118  |
| vb_q2     | 0.00164  |
------------------------
------------------------
| grad_norm | 0.038    |
| loss      | 0.0246   |
| loss_q0   | 0.079    |
| loss_q1   | 0.0161   |
| loss_q2   | 0.00427  |
| loss_q3   | 0.00224  |
| mse       | 0.0218   |
| mse_q0    | 0.0721   |
| mse_q1    | 0.0149   |
| mse_q2    | 0.00282  |
| mse_q3    | 0.000278 |
| samples   | 404      |
| step      | 100      |
| vb        | 0.0028   |
| vb_q0     | 0.00689  |
| vb_q1     | 0.00121  |
| vb_q2     | 0.00145  |
| vb_q3     | 0.00196  |
------------------------
------------------------
| grad_norm | 0.0361   |
| loss      | 0.0215   |


In [ ]:
th.save(model.state_dict(), 'imagen_new.pt')

In [ ]:
while 1:
    pass